In [21]:
import tushare as ts
import pandas as pd
import numpy as np
import os
import time
import tqdm
import talib as ta
import mplfinance as mpf
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
import shutil

# TOEKN_PATH = os.path.expanduser("~/.tushare.token")

# with open(TOEKN_PATH, "r") as f:
#     token = f.read().strip()
#     ts.set_token(token=token)
#     pro = ts.pro_api(token=token)


In [22]:
stock_basic_df = pd.read_csv("./data/stock_basic_df.csv")
ggt_df = pd.read_csv("./data/ggt_basic_df.csv")
etf_df = pd.read_csv("./data/etf_basic_df.csv")
basic_df = pd.read_csv("./data/basic_df.csv")
industry_df = pd.read_csv("./data/industry.csv")
buy_list = pd.read_csv("./data/buy_list.csv")['Column1'].tolist()
up_list = pd.read_csv("./data/up_list.csv")['Column1'].tolist()

In [23]:
# get_industry_info
hold_csv = 'table1120.csv'
hold_df = pd.read_csv(hold_csv)
hold_df = hold_df['证券代码'].apply(lambda x: str(x).zfill(6))
hold_df = hold_df.apply(lambda x: x if not isinstance(x, str) else (x + '.SH' if (x.startswith('6') or x.startswith('5')) else (x + '.SZ' if (x.startswith('0') or x.startswith('1') or x.startswith('3')) else x)))
hold_list = list(hold_df)
hold_list[0]

'001234.SZ'

In [24]:
sell_list = [i for i in hold_list if i not in up_list]
sell_list

['560010.SH', '588290.SH']

In [25]:
hold_df = pd.DataFrame()

for ts_code in hold_list:
    # 从 daily 里读 csv,只读第一行然后按 circ_mv 排序
    #df = pd.read_csv(os.path.join('./data/daily', f"{ts_code}.csv"))
    
    if ts_code not in basic_df['ts_code'].values:
        hold_df.loc[ts_code, 'name'] = str(ts_code)
        #hold_df.loc[ts_code, 'industry'] = pd.NA
        continue
    name = basic_df.loc[basic_df['ts_code'] == ts_code, 'name'].values[0]
    industry = basic_df.loc[basic_df['ts_code'] == ts_code, 'industry'].values[0]
    hold_df.loc[ts_code, 'name'] = name
    hold_df.loc[ts_code, 'industry'] = industry
print(hold_df.sort_values(by='industry'))
# 计算industry列中每个值出现的次数
industry_counts = hold_df['industry'].value_counts()
# 找出出现次数大于1的行业，即重复项
duplicates = industry_counts[industry_counts > 1].index.tolist()
duplicates

                name industry
688638.SH       誉辰智能     专用机械
603056.SH       德邦股份     仓储物流
001299.SZ       美能能源     供气供热
600096.SH        云天化     农药化肥
301212.SZ       联盛化学     化工原料
688530.SH       欧莱新材      半导体
600064.SH       南京高科     园区开发
002508.SZ       老板电器     家用电器
603280.SH       南方路机     工程机械
601068.SH       中铝国际     建筑工程
001234.SZ        泰慕士       服饰
301107.SZ       瑜欣电子     机械基件
603172.SH       万丰股份     染料涂料
001260.SZ       坤泰股份     汽车配件
601666.SH       平煤股份     煤炭开采
603588.SH       高能环境     环境保护
002546.SZ       新联电子     电气设备
002394.SZ       联发股份       纺织
688695.SH       中创股份     软件服务
600016.SH       民生银行       银行
600036.SH       招商银行       银行
001366.SZ       播恩集团       饲料
159869.SZ      游戏ETF      NaN
515790.SH      光伏ETF      NaN
516160.SH     新能源ETF      NaN
560010.SH  560010.SH      NaN
562800.SH    稀有金属ETF      NaN
588290.SH  588290.SH      NaN


['银行']

In [30]:
t_code = '600089.SH'
name = basic_df.loc[basic_df['ts_code'] == t_code, 'name'].values[0]
industry = basic_df.loc[basic_df['ts_code'] == t_code, 'industry'].values[0]
print(t_code,name,industry)

t_df = hold_df[hold_df['industry'] == industry]
t_df

600089.SH 特变电工 电气设备


,name,industry
002546.SZ,新联电子,电气设备


In [27]:
print(len(hold_df))
hold_df.sort_index()

28


,name,industry
001234.SZ,泰慕士,服饰
001260.SZ,坤泰股份,汽车配件
001299.SZ,美能能源,供气供热
001366.SZ,播恩集团,饲料
002394.SZ,联发股份,纺织
002508.SZ,老板电器,家用电器
002546.SZ,新联电子,电气设备
159869.SZ,游戏ETF,NaN
301107.SZ,瑜欣电子,机械基件
301212.SZ,联盛化学,化工原料
